In [13]:
import os
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os 
 
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.applications.mobilenet_v2 import MobileNetV2
from livelossplot.keras import PlotLossesCallback@

In [7]:
import pandas as pd 
df = pd.read_csv('vinted_data.csv')
df

,Id,Titre,Source,Couleur
0,338907174,Chemise_The_Kooples_rose_ple,https://images.vinted.net/thumbs/310x430/05908...,#E6DACD
1,400207950,T-shirt_blanc_neuf_XL,https://images.vinted.net/thumbs/310x430/06821...,#F3ECE4
2,393538053,Blouse__Haut_Grace_and_Mila_noire_taille_S,https://images.vinted.net/thumbs/310x430/057d7...,#BBBDBE
3,400227021,Chemise_Aigle,https://images.vinted.net/thumbs/310x430/05187...,#EEECED
4,398870257,T-shirt_Replay,https://images.vinted.net/thumbs/310x430/0564a...,#E2E0E0
...,...,...,...,...
24599,400783629,Robe_noire_soire_1_Cache_Cache,https://images.vinted.net/thumbs/310x430/06e56...,#C2C2C2
24600,400783485,Mini_jupe_a_sequins_rose_et_beige_Pimkie,https://images.vinted.net/thumbs/310x430/06f40...,#D1CBCC
24601,400783446,Jupe_blanche_motif_zbr,https://images.vinted.net/thumbs/310x430/06fd0...,#ECECEC
24602,400783412,Superbe_jupe_cop_copine,https://images.vinted.net/thumbs/310x430/070c4...,#C2C5C7


In [2]:
batch_size = 32

In [4]:
from keras_preprocessing.image import ImageDataGenerator

TRAIN_DIR = 'data/vinted'

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=45,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = .2)

test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split = .2)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(96,96),
    subset='training'
)

val_generator = test_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(96,96),
    subset='validation'
)

classes = len(train_generator.class_indices)
print(train_generator.class_indices)

Found 19681 images belonging to 11 classes.
Found 4915 images belonging to 11 classes.
{'Baskets': 0, 'Bottines': 1, 'Chemises': 2, 'Hauts & Tee-shirts': 3, 'Jupes': 4, 'Manteaux & vestes': 5, 'Pantalons': 6, 'Robes': 7, 'Sacs': 8, 'Sandales': 9, 'Sweats et sweats à capuche': 10}


In [5]:
label_class = ['Sacs', 'Sandales', 'Baskets', 'Pantalons', 'Hauts & Tee-shirts', 'Jupes', 'Robes', 'Chemises', 'Manteaux & vestes', 'Sweats et sweats à capuche', 'Bottines']

In [15]:
label_class = ['Sacs', 'Sandales', 'Baskets', 'Pantalons', 'Hauts & Tee-shirts', 'Jupes', 'Robes', 'Chemises', 'Manteaux & vestes', 'Sweats et sweats à capuche', 'Bottines']

TRAIN_DIR = 'data/vinted'

"""train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=45,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = .2)

test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split = .2)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(96,96),
    subset='training'
)

val_generator = test_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(96,96),
    subset='validation'
)

classes = len(train_generator.class_indices)
print(train_generator.class_indices)


"""

# create the base pre-trained model
base_model = MobileNetV2(input_shape=(96, 96, 3),classes=11, include_top=False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(11, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
print(len(model.layers))
for layer in model.layers[:120]:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','mae', 'categorical_accuracy'])
model.summary()
                 
                 
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from math import ceil

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=ceil(0.8 * (df.size / batch_size)),

    validation_data=val_generator,
    validation_steps=ceil(0.2 * (df.size / batch_size)),
    callbacks=[PlotLossesCallback()],
    epochs=90,
    shuffle=True,
    verbose=1
)
history

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

158
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 97, 97, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
________________________________________________________________________________________

Epoch 1/90
   3/2461 [..............................] - ETA: 3:26:59 - loss: 6.8493 - accuracy: 0.1146 - mae: 0.1594 - categorical_accuracy: 0.1146

KeyboardInterrupt: 